In [ ]:
%load_ext rpy2.ipython


In [ ]:
%%R

# Ajustamos el modelo usando bedrooms como variable independiente
modelo <- lm(price ~ bedrooms, data = departamentos)

# Resumen del modelo
summary(modelo)


In [ ]:
%%R

# Departamento de 2 dormitorios
coef(modelo)[1] + coef(modelo)[2] * 2


In [ ]:
%%R

# Predicciones para todas las observaciones en la data
departamentos_in_regression <- departamentos[!is.na(departamentos$bedrooms) & !is.na(departamentos$price), ]
predicciones <- predict(modelo)
head(predicciones)


In [ ]:
%%R

# Ploteamos predicción con ggplot2
ggplot(departamentos_in_regression, aes(x = bedrooms, y = predicciones)) +
  geom_point() +
  labs(x = "Dormitorios", y = "Precio predicho", 
       title = "Predicciones vs. Número de dormitorios") +
  theme_minimal()


In [ ]:
%%R

# Calculamos MSE
mse <- sum(residuals(modelo)^2) / (length(residuals(modelo)) - 2)
mse


In [ ]:
%%R

# Raíz cuadrada del MSE
sqrt(mse)


In [ ]:
%%R

# Histograma de residuos
ggplot(data.frame(residuos = residuals(modelo)), aes(x = residuos)) +
  geom_histogram(bins = 50) +
  labs(title = "Distribución de los residuos",
       x = "Residuos",
       y = "Frecuencia") +
  theme_minimal()


In [ ]:
%%R

# Agregamos los residuos al dataframe
departamentos_in_regression$residuos <- residuals(modelo)

# Gráfico de residuos vs. variable independiente
ggplot(departamentos_in_regression, aes(x = bedrooms, y = residuos)) +
  geom_point() +
  geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
  labs(title = "Residuos vs. Número de dormitorios",
       x = "Dormitorios",
       y = "Residuos") +
  theme_minimal()


In [ ]:
%%R

# Mapa de residuos
ggplot() +
  geom_sf(data = departamentos_in_regression, aes(color = residuos)) +
  scale_color_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +
  labs(title = "Distribución espacial de residuos",
       color = "Residuos") +
  theme_minimal()


In [ ]:
%%R

# Instalamos los paquetes necesarios si no están instalados
if (!require("sf")) install.packages("sf")
if (!require("dplyr")) install.packages("dplyr")
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("car")) install.packages("car")  # Para diagnósticos de regresión
if (!require("lmtest")) install.packages("lmtest")  # Para pruebas de hipótesis
if (!require("sandwich")) install.packages("sandwich")  # Para errores estándar robustos


In [ ]:
%%R

# Cargamos las librerías
library(sf)
library(dplyr)
library(ggplot2)
library(car)
library(lmtest)
library(sandwich)


In [ ]:
%%R

# Descargamos y cargamos los datos
download.file("https://github.com/rpasquini/urban-econometrics/blob/master/data/departamentos2019amba.zip?raw=true",
              destfile = "departamentos2019amba.zip", mode = "wb")
unzip("departamentos2019amba.zip", exdir = ".")
departamentos <- st_read("departamentos2019amba.shp")


In [ ]:
%%R

# Gráfico de dispersión
ggplot(departamentos, aes(x = surface_to, y = price)) +
  geom_point(alpha = 0.5) +
  geom_smooth(method = "lm", color = "red") +
  labs(x = "Superficie (m²)", y = "Precio", title = "Relación entre Precio y Superficie") +
  theme_minimal()


In [ ]:
%%R

# Ajustamos el modelo
modelo <- lm(price ~ surface_to, data = departamentos)

# Resumen del modelo
summary(modelo)


In [ ]:
%%R

# Gráficos de diagnóstico
par(mfrow = c(2, 2))
plot(modelo)


In [ ]:
%%R

# Test de Breusch-Pagan para heterocedasticidad
bptest(modelo)

# Test de Durbin-Watson para autocorrelación
dwtest(modelo)

# Intervalos de confianza para los coeficientes
confint(modelo)


In [ ]:
%%R

# Calculamos errores estándar robustos
coeftest(modelo, vcov = vcovHC(modelo, type = "HC1"))


In [ ]:
%%R

# Crear algunos valores de superficie para predecir
nuevos_datos <- data.frame(surface_to = c(50, 75, 100))

# Realizar predicciones
predicciones <- predict(modelo, newdata = nuevos_datos, interval = "confidence")

# Mostrar resultados
cbind(nuevos_datos, predicciones)


In [ ]:
%%R

# Gráfico con predicciones e intervalos de confianza
ggplot(departamentos, aes(x = surface_to, y = price)) +
  geom_point(alpha = 0.5) +
  geom_smooth(method = "lm", color = "red") +
  labs(x = "Superficie (m²)", y = "Precio", 
       title = "Modelo de regresión con intervalos de confianza") +
  theme_minimal()
